# IRENE-3338

In [ ]:
import datetime

In [ ]:
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math


In [ ]:
from invisible_cities.database import load_db
from   invisible_cities.core.system_of_units_c import units
import invisible_cities.sierpe.blr as blr
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.reco.wfm_functions as wfm
import invisible_cities.reco.tbl_functions as tbl
from invisible_cities.reco.params import S12Params, ThresholdParams, CalibratedSum, PMaps
import invisible_cities.reco.peak_functions as pf
import invisible_cities.reco.peak_functions_c as cpf
import invisible_cities.reco.pmaps_functions as pmf
from   invisible_cities.cities.base_cities import S12Params as S12P

In [ ]:
from invisible_cities.core.core_functions import define_window
from invisible_cities.core.core_functions import lrange
from collections import namedtuple

In [ ]:
from invisible_cities.cities.irene import Irene, IRENE

In [ ]:
%load_ext Cython

## Files

In [ ]:
RWF_file = os.path.join(os.environ['IC_DATA'],'LSC/dst_waves_f0_3338.root.h5')

In [ ]:
h5rwf = False

In [ ]:
if h5rwf:
    h5rwf.close()

In [ ]:
h5rwf = tb.open_file(RWF_file,'r')

In [ ]:
h5rwf

### pmtrwf and sipmrwf vectors

To get vectors use **get_vectors(file)** in tbl_functions

In [ ]:
pmtrwf = h5rwf.root.RD.pmtrwf
sipmrwf = h5rwf.root.RD.sipmrwf

In [ ]:
pmt_active = list(range(11))

In [ ]:
Pmaps= namedtuple('Pmaps', 'S1 S2 S2Si S1p PMT_S1 PMT_S1p')

In [ ]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

### Access to data base and definition of units

In [ ]:
DataPMT = load_db.DataPMT(3338)
adc_to_pes = abs(DataPMT.adc_to_pes.values)
coeff_c = abs(DataPMT.coeff_c.values)
coeff_blr = abs(DataPMT.coeff_blr.values)
DataSiPM = load_db.DataSiPM()
adc_to_pes_sipm = DataSiPM.adc_to_pes.values
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

In [ ]:
DataPMT

In [ ]:
coeff_blr = np.array([1.61,
                      1.61,
                      1.61,
                      1.61,
                      1.60,
                      0.8,
                      0.8,
                      0.8,
                      0.8,
                      0.8,
                      1.61,
                      0.8]) * 0.001

In [ ]:
coeff_c = np.array([2.94,
                    2.74,
                    3.08,
                    2.81,
                    2.88,
                    1.,
                    1.,
                    1.,
                    1.,
                    1.,
                    2.75,
                    1.0]) * 1e-6

In [ ]:
adc_to_pes = np.array([24.3,
                    26.2,
                    22.19,
                    19.55,
                    33.88,
                    25.8,
                    21.8,
                    24.8,
                    26.4,
                    22.4,
                    23.3,
                    26.7]) 

In [ ]:
coeff_blr

### Parameters for peak search

In [ ]:
s1par = S12P(tmin=0*units.mus, tmax=590*units.mus, lmin=6, lmax=20, stride=4, rebin=False)
s2par = S12P(tmin=590*units.mus, tmax=690*units.mus, lmin=80, lmax=200000, stride=40, rebin=True)
s1p    = S12P(tmin=620*units.mus, tmax=1100*units.mus, lmin=6, lmax=20, stride=4, rebin=False)
thresholds = ThresholdParams(thr_s1=0.5*units.pes, 
                     thr_s2=1*units.pes, 
                     thr_MAU=3*units.adc, 
                     thr_sipm=5*units.pes, 
                     thr_SIPM=20*units.pes )
s1par_PMT = S12P(tmin=0*units.mus, tmax=590*units.mus, lmin=4, lmax=20, stride=4, rebin=False)
thresholds_PMT = ThresholdParams(thr_s1=0.1*units.pes, 
                     thr_s2=1*units.pes, 
                     thr_MAU=3*units.adc, 
                     thr_sipm=5*units.pes, 
                     thr_SIPM=20*units.pes )

### Calibrated sum

In [ ]:
%%cython
cimport numpy as np
import  numpy as np
from scipy import signal

cpdef calibrated_pmt_sum(double [:, :]  CWF,
                         double [:]     adc_to_pes,
                         list           pmt_active = [],
                         int            n_MAU = 200,
                         double         thr_MAU =   5):
    """
    Computes the ZS calibrated sum of the PMTs
    after correcting the baseline with a MAU to suppress low frequency noise.
    input:
    CWF:    Corrected waveform (passed by BLR)
    adc_to_pes: a vector with calibration constants
    n_MAU:  length of the MAU window
    thr_MAU: treshold above MAU to select sample

    """

    cdef int j, k
    cdef int NPMT = CWF.shape[0]
    cdef int NWF  = CWF.shape[1]
    cdef double [:] MAU = np.array(np.ones(n_MAU),
                                   dtype = np.double) * (1 / n_MAU)


    # CWF if above MAU threshold
    cdef double [:, :] pmt_thr  = np.zeros((NPMT,NWF), dtype=np.double)
    cdef double [:]    csum     = np.zeros(      NWF , dtype=np.double)
    cdef double [:]    csum_mau = np.zeros(      NWF , dtype=np.double)
    cdef double [:]    MAU_pmt  = np.zeros(      NWF , dtype=np.double)
    
    cdef list PMT = list(range(NPMT))
    if len(pmt_active) > 0:
        PMT = pmt_active
    for j in PMT:
        # MAU for each of the PMTs, following the waveform
        MAU_pmt = signal.lfilter(MAU, 1, CWF[j,:])

        for k in range(NWF):
            if CWF[j,k] >= MAU_pmt[k] + thr_MAU: # >= not >: found testing!
                pmt_thr[j,k] = CWF[j,k]

    for j in PMT:
        for k in range(NWF):
            csum_mau[k] += pmt_thr[j, k] * 1 / adc_to_pes[j]
            csum[k] += CWF[j, k] * 1 / adc_to_pes[j]

    return np.asarray(csum), np.asarray(csum_mau)

cpdef calibrated_pmt_mau(double [:, :]  CWF,
                         double [:]     adc_to_pes,
                         list           pmt_active = [],
                         int            n_MAU = 200,
                         double         thr_MAU =   5):
    """
    Returns the calibrated waveforms for PMTs correcting by MAU.
    input:
    CWF:    Corrected waveform (passed by BLR)
    adc_to_pes: a vector with calibration constants
    n_MAU:  length of the MAU window
    thr_MAU: treshold above MAU to select sample

    """

    cdef int j, k
    cdef int NPMT = CWF.shape[0]
    cdef int NWF  = CWF.shape[1]
    cdef list PMT = list(range(NPMT))
    if len(pmt_active) > 0:
        PMT = pmt_active

    
    cdef double [:] MAU = np.array(np.ones(n_MAU),
                                   dtype = np.double) * (1 / n_MAU)
    
    # CWF if above MAU threshold
    cdef double [:, :] pmt_thr  = np.zeros((NPMT,NWF), dtype=np.double)
    cdef double [:]    MAU_pmt  = np.zeros(      NWF , dtype=np.double)
           
    for j in PMT:
        # MAU for each of the PMTs, following the waveform
        MAU_pmt = signal.lfilter(MAU, 1, CWF[j,:])

        for k in range(NWF):
            if CWF[j,k] >= MAU_pmt[k] + thr_MAU: # >= not >: found testing!
                pmt_thr[j,k] = CWF[j,k] * 1 / adc_to_pes[j]


    return np.asarray(pmt_thr)

In [ ]:
Csum= namedtuple('Csum', 'csum csum_mau cpmt')

In [ ]:
def compute_csum_and_pmaps(pmtrwf, sipmrwf, pmt_active, 
                           coeff_c, coeff_blr, adc_to_pes, 
                           s1_params, s2_params, s1p_params, thresholds,
                           s1par_PMT, thresholds_PMT,
                           event, run_number=0):
    """Compute calibrated sum and PMAPS.

    :param pmtrwf: PMTs RWF
    :param sipmrwf: SiPMs RWF
    :param s1par: parameters for S1 search (S12Params namedtuple)
    :param s2par: parameters for S2 search (S12Params namedtuple)
    :param thresholds: thresholds for searches (ThresholdParams namedtuple)
    :param event: event number

    :returns: a nametuple of calibrated sum and a namedtuple of PMAPS
    """

    thr = thresholds
    thr2 = thresholds_PMT

    # data base
    #DataPMT = load_db.DataPMT(run_number)
    #adc_to_pes = abs(DataPMT.adc_to_pes.values)
    #coeff_c    = abs(DataPMT.coeff_c   .values)
    #coeff_blr  = abs(DataPMT.coeff_blr .values)
    DataSiPM   = load_db.DataSiPM()
    adc_to_pes_sipm = DataSiPM.adc_to_pes.values

    # deconv
    CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)

    # calibrated sum
    csum, csum_mau = calibrated_pmt_sum(CWF, adc_to_pes, pmt_active, 
                                        n_MAU=100, thr_MAU=thr.thr_MAU)
    
    # calibrated PMT waveforms (above MAU)
    CPMT           = calibrated_pmt_mau(CWF, adc_to_pes, pmt_active, 
                                        n_MAU=100, thr_MAU=0.)
    
    # S2
    s2_ene, s2_indx = cpf.wfzs(csum, threshold=thr.thr_s2)
    S2 =  cpf.find_S12(s2_ene, s2_indx, **s2_params._asdict())
    
    # S1 and S1p
    s1_ene, s1_indx = cpf.wfzs(csum_mau, threshold=thr.thr_s1)
    S1 =  cpf.find_S12(s1_ene, s1_indx, **s1_params._asdict())
    S1p = cpf.find_S12(s1_ene, s1_indx, **s1p_params._asdict())
    
    # S1 and S1p for individual PMTs
    if len(S2) == 1:
        t = S2[0][0]
        tmin = t[-1] + 1*units.mus
        s1p    = S12P(tmin=tmin, tmax=1100*units.mus, 
                      lmin=6, lmax=20, stride=4, rebin=False)
    PMT_S1 = {}
    PMT_S1p = {}
    for pmt in pmt_active:
        s1_ene, s1_indx = cpf.wfzs(CPMT[pmt], threshold=thr2.thr_s1)
        PMT_S1[pmt] = cpf.find_S12(s1_ene, s1_indx, **s1par_PMT._asdict())
        PMT_S1p[pmt] = cpf.find_S12(s1_ene, s1_indx, **s1par_PMT._asdict())

    sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=thr.thr_sipm, n_MAU=100)
    SIPM = cpf.select_sipm(sipm)
    S2Si = pf.sipm_s2_dict(SIPM, S2, thr=thr.thr_SIPM)
    S2Si = 0
    return (Csum(csum=csum, csum_mau=csum_mau, cpmt=CPMT),
            Pmaps(S1=S1, S2=S2, S2Si=S2Si, S1p=S1p, PMT_S1=PMT_S1, PMT_S1p=PMT_S1p))

In [ ]:
def plot_pmt_signals_vs_time_mus(pmt_signals,
                                 pmt_active,
                                 t_min      =    0,
                                 t_max      = 1200,
                                 signal_min =    0,
                                 signal_max =  200):
    """Plot all the PMT signals versus time in mus (tmin, tmax in mus)."""

    tstep = 25
    PMTWL = pmt_signals[0].shape[0]
    signal_t = np.arange(0., PMTWL * tstep, tstep)/units.mus
    plt.figure(figsize=(10, 10))
    j=0
    for i in pmt_active:
        ax1 = plt.subplot(4, 3, j+1)
        ax1.set_xlim([t_min, t_max])
        ax1.set_ylim([signal_min, signal_max])
        mpl.set_plot_labels(xlabel = "t (mus)",
                        ylabel = "signal (pes/adc)")

        plt.plot(signal_t, pmt_signals[i])
        j+=1

In [ ]:
def plot_signal_vs_time_mus(signal,
                            t_min      =    0,
                            t_max      = 1200,
                            signal_min =    0,
                            signal_max =  200,
                            label=''):
    """Plot signal versus time in mus (tmin, tmax in mus). """
    tstep = 25 # in ns
    PMTWL = signal.shape[0]
    signal_t = np.arange(0., PMTWL * tstep, tstep)/units.mus
    ax1 = plt.subplot(1, 1, 1)
    ax1.set_xlim([t_min, t_max])
    ax1.set_ylim([signal_min, signal_max])
    mpl.set_plot_labels(xlabel = "t (mus)",
                    ylabel = "signal (pes/adc)")
    plt.plot(signal_t, signal, label=label)
    legend = plt.legend(loc='upper right')
    for label in legend.get_texts():
        label.set_fontsize('small')

In [ ]:
def plot_csum_in_mus(CSUM, tmin=0, tmax=1200, signal_min=-5, signal_max=200, 
                     csum=True, csum_mau=False):
    """Plots in mus (notice units)"""
    
    if csum:
        plot_signal_vs_time_mus(CSUM.csum, 
                                t_min=tmin, t_max=tmax, 
                                signal_min=signal_min, signal_max=signal_max,
                                label='CSUM')
    if csum_mau:
        plot_signal_vs_time_mus(CSUM.csum_mau, 
                                t_min=tmin, t_max=tmax, 
                                signal_min=signal_min, signal_max=signal_max,
                                label='CSUM_MAU')

### RWF

In [ ]:
event=10
RWF = pmtrwf[event]

In [ ]:
wfm.plot_pmt_waveforms(RWF, zoom=True, window_size=10000)

In [ ]:
coeff_blr

In [ ]:
coeff_c

In [ ]:
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=38000, thr_trigger=5)

In [ ]:
wfm.plot_pmt_waveforms(CWF, zoom=True, window_size=20000)

In [ ]:
plot_pmt_signals_vs_time_mus(CWF,
                             pmt_active,
                             t_min      =    0,
                             t_max      = 1200,
                             signal_min =    -5,
                             signal_max =  50)

In [ ]:
plot_signal_vs_time_mus(CWF[0],
                            t_min      =    0,
                            t_max      = 1200,
                            signal_min =    0,
                            signal_max =  200,
                            label='plot')

### NB: 
1. Huge S1-like near 400
2. Very small S2 (cannot correspond to "S1")
3. Waveform clean before S2
4. Pseudo "electron rain" (ER) after S2. Different in each channel. Clearest effect in channel 9.

In [ ]:
csum, csum_mau = calibrated_pmt_sum(CWF,adc_to_pes,pmt_active)

In [ ]:
plot_signal_vs_time_mus(csum, t_min=0, t_max=1200, signal_min=-5, signal_max=250, label='CSUM')

In [ ]:
plot_signal_vs_time_mus(csum, t_min=590, t_max=620, signal_min=-5, signal_max=250, label='CSUM')

In [ ]:
plot_signal_vs_time_mus(csum_mau, t_min=559, t_max=561, signal_min=-5, signal_max=10, label='CSUM')

### NB:
1. Very small signal.
2. calibrated sum appears "cleaner" than individual WFs.

In [ ]:
CSUM, P = compute_csum_and_pmaps(pmtrwf, sipmrwf, pmt_active, 
                                 coeff_c, coeff_blr, adc_to_pes, 
                                 s1par, s2par, s1p, thresholds, 
                                 s1par_PMT, thresholds_PMT, 
                                 event=event, run_number=0)

In [ ]:
plot_pmt_signals_vs_time_mus(CSUM.cpmt,
                             pmt_active,
                             t_min      =    559,
                             t_max      = 561,
                             signal_min =    -3,
                             signal_max =  3)

In [ ]:
plot_pmt_signals_vs_time_mus(CSUM.cpmt,
                             pmt_active,
                             t_min      =   0,
                             t_max      = 1200,
                             signal_min =    -3,
                             signal_max =  30)

In [ ]:
P.S1

In [ ]:
P.S2

In [ ]:
pmf.scan_s12(P.S1)

In [ ]:
pmf.scan_s12(P.S2)

In [ ]:
P.S1p

In [ ]:
pmf.scan_s12(P.S1p)

In [ ]:
P.PMT_S1

In [ ]:
P.PMT_S1p

In [ ]:
pmf.scan_s12(P.S1)
pmf.scan_s12(P.PMT_S1[1])
pmf.scan_s12(P.PMT_S1[2])
pmf.scan_s12(P.PMT_S1[7])

In [ ]:
 pmf.scan_s12(P.S1)

In [ ]:
 pmf.scan_s12(P.S2)

In [ ]:
P.PMT_S1

In [ ]:
def compare_S1(P):
    n_match_s1 = 0
    for peak, (t,E) in P.S1.items():
        for pmt in P.PMT_S1:
            if len (P.PMT_S1[pmt]) > 0:
                for peak2, (t2,E2) in P.PMT_S1[pmt].items():
                    if abs(t2[0] - t[0]) < 100*units.mus:
                        print('for pmt = {}'.format(pmt))
                        print('found mach between S1 peak = {} and S1_PMT peak = {}'.\
                              format(peak, peak2))
                        print('S1 t = {}'.format(t))
                        print('PMT_ S1 t = {}'.format(t2))
                        if peak == peak2:
                            n_match_s1 +=1
    return n_match_s1
        

In [ ]:
P.S1

In [ ]:
n_match_s1 = compare_S1(P)

In [ ]:
n_match_s1

In [ ]:
P.S1p

In [ ]:
event=1
RWF = pmtrwf[event]
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=38000, thr_trigger=5)

In [ ]:
plot_pmt_signals_vs_time_mus(CWF,
                             pmt_active,
                             t_min      =    0,
                             t_max      = 1200,
                             signal_min =    -5,
                             signal_max =  150)

#### NB
1. all PMTs see someting near 100 (shining?)
2. seems to be a real S1 near 500.
3. S1 appears too big for S2.
4. ER now apparent in almost all PMTs. Different pattern. 

In [ ]:
CSUM, P = compute_csum_and_pmaps(pmtrwf, sipmrwf, pmt_active, s1par, s2par, s1p, 
                                 thresholds, thresholds_PMT, event=event, run_number=0)

In [ ]:
CSUM, P = compute_csum_and_pmaps(pmtrwf, sipmrwf, pmt_active, s1par, s2par, s1p, thresholds, event=event, run_number=0)

In [ ]:
n_match_s1 = compare_S1(P)

### NB, ER now visible.

In [ ]:
plot_csum_in_mus(CSUM, tmin=200, tmax=500, signal_min=-5, signal_max=5, csum=True, csum_mau=True)

In [ ]:
plot_csum_in_mus(CSUM, tmin=650, tmax=1200, signal_min=-5, signal_max=5, csum=True, csum_mau=True)

In [ ]:
P.S1

In [ ]:
P.PMT_S1

In [ ]:
 pmf.scan_s12(P.S1)

In [ ]:
 pmf.scan_s12(P.S2)

In [ ]:
 pmf.scan_s12(P.S1p)

In [ ]:
P.S1p

In [ ]:
P.PMT_S1p

### NB, Individual PMTs are effective to tag true S1s

In [ ]:
def plot_event(event):

    RWF = pmtrwf[event]
    CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=38000, thr_trigger=5)
    CSUM, P = compute_csum_and_pmaps(pmtrwf, sipmrwf, pmt_active, s1par, s2par, s1p, thresholds, event=event, run_number=0)
    plot_pmt_signals_vs_time_mus(CWF,
                             pmt_active,
                             t_min      =    0,
                             t_max      = 1200,
                             signal_min =    -5,
                             signal_max =  150)
    plt.show()
    plot_csum_in_mus(CSUM, tmin=0, tmax=1200, signal_min=-5, signal_max=20, csum=True, csum_mau=False)
    plt.show()
    plot_csum_in_mus(CSUM, tmin=200, tmax=500, signal_min=-5, signal_max=5, csum=True, csum_mau=True)
    plt.show()
    plot_csum_in_mus(CSUM, tmin=650, tmax=1200, signal_min=-5, signal_max=5, csum=True, csum_mau=True)
    plt.show()
    pmf.scan_s12(P.S1)
    plt.show()
    pmf.scan_s12(P.S2)
    plt.show()
    pmf.scan_s12(P.S1p)   
    plt.show()

In [ ]:
plot_event(event=2)

### NB, electron rain present in this event, S1 suspiciously close to previous event

In [ ]:
plot_event(event=4)

In [ ]:
plot_event(event=5)

In [ ]:
plot_event(event=6)

In [ ]:
plot_event(event=7)

In [ ]:
plot_event(event=9)

In [ ]:
plot_event(event=10)

### To conclude, the ER is still there but the effect seems less hard than in the past and is not seen before the signal. 

In [ ]:
def s12_stats(S1L, S1pL, S2):
    n_s1 = []
    n_s1p = []
    n_s2 = []
    
    for i in S1L:
        n_s1.append(len(S1L[i]))
    for i in S1pL:
        n_s1p.append(len(S1pL[i]))
    for i in S2L:
        n_s2.append(len(S2L[i]))
    
    return np.array(n_s1), np.array(n_s1p), np.array(n_s2)

In [ ]:
n_s1, n_s1p, n_s2 = s12_stats(S1L, S1pL, S2)

In [ ]:
plt.hist(n_s1)

In [ ]:
plt.hist(n_s2)

In [ ]:
plt.hist(n_s1p)